### 标量梯度自动计算
通过`requires_grad=True`，torch将会记录该张量所有操作，可以在必要的时候通过调用`.backward()`得到梯度值。  
然后在需要计算梯度时，调用`.backward()`，torch将会自动计算梯度，最终保存在`.grad`属性中。  
如果不需要让torch在记录操作计算梯度，则可以调用`.detach()`方法停止torch的记录。

In [2]:
import torch
x = torch.ones(2, 2, requires_grad=True)
y = x + 2
z = y * y * 3
out = z.mean()
out.backward()
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


当对x标记`requires_grad=True`时，则所有对x的计算将会被追踪，上述计算中，最终$out=\frac{3}{4}(x+2)^2$，当我们调用`out.backward()`时，pytorch就开始对每一步x的计算进行求导，比如$\frac{dout}{dx}=\frac{3}{2}(x+2)$，实际上**对于标量（out）来说**，当调用`out.backward()`，等同于`out.backward(torch.tensor(1.))`

### 向量梯度自动计算

我们看一个复杂的求导

In [7]:
import torch
# 注意只有float类型的tensor才能自动求导
x = torch.arange(0,3, requires_grad=True, dtype=torch.float)
y = x**2 + x*2
z = y.sum()
z.backward()
x.grad

tensor([2., 4., 6.])

In [13]:
import torch
# 注意只有float类型的tensor才能自动求导
x = torch.arange(0,3, requires_grad=True, dtype=torch.float)
y = x**2 + x*2
z = y.sum()
y.backward(torch.tensor([1.,1.,1.]))
x.grad

tensor([2., 4., 6.])

这里注意到，我们通过`z.backward()`和`y.backward(torch.tensor([1.,1.,1.]))`得到的$\frac{dz}{dx}$结果是一样的，那这里关键就是`backward`函数的参数了。  
```python
variable.backward(grad_variables=None, retain_graph=None,create_graph=None)
```
其中，`grad_variables`：其形状与`variable`一致，对于`z.backward()`，`grad_variables`相当于链式法则$\frac{dz}{dx}=\frac{dz}{dx} \times grad\_variables$，那从上面的式子中我们可以知道，$\frac{dz}{dx}=\frac{dy}{dx}$，因此我们传入[1.0,1.0]

上一节，我们得到的out是一个数，大多数情况下，我们会得到一个向量，那一个向量对一个向量求导的一个基本公式就是雅克比矩阵，对于$\vec y=f(\vec x)$来说，我们有$$\frac{dy}{dx}=\left[
 \begin{matrix}
   \frac{\partial y_1}{\partial x_1} & \frac{\partial y_1}{\partial x_2} & \frac{\partial y_1}{\partial x_3} \\
   \frac{\partial y_2}{\partial x_1} & \frac{\partial y_2}{\partial x_2} & \frac{\partial y_2}{\partial x_3} \\
   \frac{\partial y_3}{\partial x_1} & \frac{\partial y_3}{\partial x_2} & \frac{\partial y_3}{\partial x_3} \\
  \end{matrix}
  \right] \tag{3}$$
但是实际上，pytorch不会为你自动计算这个雅克比矩阵，我们看一个例子：

In [9]:
import torch
x = torch.tensor([[3., 4.]], dtype=torch.float, requires_grad=True)
y = torch.zeros(1,2)
y[0,0] = x[0,0] ** 2 + x[0, 1]
y[0,1] = x[0,1] ** 3 + x[0, 0]
out = 2 * y
out.backward(torch.tensor([[1.,1.]]), retain_graph=True)
print(x.grad)

tensor([[14., 98.]])


上述代码转为数学公式为：
$$
x = \left[
 \begin{matrix}
  3 & 4 \\
  \end{matrix}
  \right]  \\
  y = \left[
 \begin{matrix}
  0 \\
   0
  \end{matrix}
  \right] \\
y =  \left[ 
\begin{matrix}
  x[0,0]^2+x[0,1] \\
 x[0,1]^3 + x[0,0]
  \end{matrix}
  \right] \\
  out = 2y
$$

上述代码第7行，如果我们直接调用`out.backward`，会直接报错。默认情况下，如果`out.backward()`参数为None，则只能是标量对矢量求导，在上式中$$
x=\left[
 \begin{matrix}
  x_1 & x_2 \\
  \end{matrix}
  \right]
$$

$$  
out=\left[
 \begin{matrix}
  2x_1^2 + 2x_2\\
  2x_2^3 + 2x_1 \\
  \end{matrix}
  \right] \\
\frac{dout}{dx}=\left[
 \begin{matrix}
  4x_1 & 2 \\
   2 & 6x_2^2
  \end{matrix}
  \right]
$$
我们将x代入上式可得
$$  
\frac{dout}{dx}=\left[
 \begin{matrix}
  12 & 2 \\
  2 & 96
  \end{matrix}
  \right]
$$
按照推导，我们应该得出$ 
\frac{dout}{dx}=\left[
 \begin{matrix}
  12 & 2 \\
  2 & 96
  \end{matrix}
  \right]
$这个结论，可是实际pytorch给的结果是$ 
\frac{dout}{dx}=\left[
 \begin{matrix}
  14 & 98 \\
  \end{matrix}
  \right]
$，在观察，我们给`out.backward`传的参数是`torch.tensor([[1.,1.]])`，**它和`out`维度一样**，且我们发现这两个$\frac{dy}{dx} * torch.tensor([[1.,1.]])$刚好是pytorch给的结果。  
所以如果需要完整的雅克比矩阵，那就需要我们手动传参了：
```python

```

In [28]:
import copy
x.grad.zero_()
out.backward(torch.tensor([[1.,0.]]), retain_graph=True)
g1 = copy.deepcopy(x.grad)
x.grad.zero_()
out.backward(torch.tensor([[0.,1.]]), retain_graph=True)
g2 = copy.deepcopy(x.grad)
torch.cat((g1, g2), 0)

tensor([[12.,  2.],
        [ 2., 96.]])

可以看到我们得到了完成的雅克比矩阵，这里有些细节要注意：
1. retain_graph，pytorch是将所有操作构成图，每次计算图之前节点就会舍弃，所以这里要保留，否则下一次backward就会报错
2. zero_() 是为了将之前图中节点信息清零，便于重新计算

参考：[pytoch求导-博客园](https://www.cnblogs.com/JeasonIsCoding/p/10164948.html)